# I27 : Prefer Public Attributes Over Private Ones

In [1]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field

In [2]:
foo = MyObject()
assert foo.public_field == 5

In [3]:
assert foo.get_private_field() == 10

> cannot enter directly to \_\_ attr

In [4]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

In [11]:
class MyOtherObject(object):
    def __init___(self):
        self.__private_field = 71
        
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

In [12]:
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

AttributeError: 'MyOtherObject' object has no attribute '_MyOtherObject__private_field'

In [14]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
        

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

In [16]:
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

- As	you’d	expect	with	private	fields,	a	subclass	can’t	access	its	parent	class’s	private	fields.

<pre>

The	private	attribute	behavior	is	implemented	with	a	simple	transformation	of	the	attribute name.	When	the	Python	compiler	sees	private	attribute	access	in	methods	like MyChildObject.get_private_field,	it	translates	__private_field	to access	_MyChildObject__private_field	instead.	In	this	example, __private_field	was	only	defined	in	MyParentObject.__init__,	meaning the	private	attribute’s	real	name	is	_MyParentObject__private_field. Accessing	the	parent’s	private	attribute	from	the	child	class	fails	simply	because	the transformed	attribute	name	doesn’t	match. 

</pre>

- Knowing	this	scheme,	you	can	easily	access	the	private	attributes	of	any	class,	from	a subclass	or	externally,	without	asking	for	permission. 

In [17]:
assert baz._MyParentObject__private_field == 71

In [18]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


- Why	doesn’t	the	syntax	for	private	attributes	actually	enforce	strict	visibility?	The simplest	answer	is	one	often-quoted	motto	of	Python:	“We	are	all	consenting	adults	here.” Python	programmers	believe	that	the	benefits	of	being	open	outweigh	the	downsides	of being	closed. 

In [19]:
class MyClass(object):
    def __init__(self, value):
        self.__value = value
        
    def get_value(self):
        return str(self.__value)

In [20]:
foo = MyClass(5)
assert foo.get_value() == '5'

- **This	is	the	wrong	approach.**	Inevitably	someone,	including	you,	will	want	to	subclass	your class	to	add	new	behavior	or	to	work	around	deficiencies	in	existing	methods	(like	above, how	MyClass.get\_value	always	returns	a	string).	By	choosing	private	attributes, you’re	only	making	subclass	overrides	and	extensions	cumbersome	and	brittle.	Your potential	subclassers	will	still	access	the	private	fields	when	they	absolutely	need	to	do	so.

In [21]:
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [22]:
foo = MyIntegerSubclass(5)
assert foo.get_value() == 5

- But	if	the	class	hierarchy	changes	beneath	you,	these	classes	will	break	because	the	private references	are	no	longer	valid.	Here,	the	MyIntegerSubclass	class’s	immediate parent,	MyClass,	has	had	another	parent	class	added	called	MyBaseClass

In [24]:
class MyBaseClass(object):
    def __init__(self, value):
        self.__value = value
    #...
    
class MyClass(MyBaseClass):
    # ....
    pass

class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

- The	\_\_value	attribute	is	now	assigned	in	the	MyBaseClass	parent	class,	not	the MyClass	parent.	That	causes	the	private	variable	reference	self.\_MyClass\_\_value to	break	in	MyIntegerSubclass. 

In [25]:
foo = MyIntegerSubclass(5)
foo.get_value()

AttributeError: 'MyIntegerSubclass' object has no attribute '_MyClass__value'

- In	general,	**it’s	better	to	err	on	the	side	of	allowing	subclasses	to	do	more	by	using protected	attributes.**	Document	each	protected	field	and	explain	which	are	internal	APIs available	to	subclasses	and	which	should	be	left	alone	entirely.	This	is	as	much	advice	to other	programmers	as	it	is	guidance	for	your	future	self	on	how	to	extend	your	own	code safely. 

In [27]:
class MyClass(object):
    def __init__(self, value):
        # This stores the user-supplied value for the object.
        # It should be coercible to a string. Once assigned for 
        # the object it should be treated as immutable
        self._value = value

- The	only	time	to	seriously	consider	using	private	attributes	is	**when	you’re	worried	about naming	conflicts	with	subclasses.**	This	problem	occurs	when	a	child	class	unwittingly defines	an	attribute	that	was	already	defined	by	its	parent	class. 

In [28]:
class ApiClass(object):
    def __init__(self):
        self.value = 5
        
    def get(self):
        return self._value
    

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

In [29]:
a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


- This	is	primarily	a	concern	with	classes	that	are	part	of	a	public	API;	the	subclasses	are out	of	your	control,	so	you	can’t	refactor	to	fix	the	problem.	Such	a	conflict	is	especially possible	with	attribute	names	that	are	very	common	(like	value).	To	reduce	the	risk	of this	happening,	you	can	use	a	private	attribute	in	the	parent	class	to	ensure	that	there	are no	attribute	names	that	overlap	with	child	classes. 

In [30]:
class ApiClass(object):
    def __init__(self):
        self.__value = 5
        
    def get(self):
        return self.__value
    

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'

In [32]:
a = Child() 
print(a.get(), 'and', a._value, 'are different')

5 and hello are different


# I28 : Inherit from collections.abc for Custom Container Types

- what if we need to make custom list that has additional methods for counting the frequency of its members?

In [34]:
class FrequencyList(list):
    def __init__(self, members):
        super().__init__(members)

    def frequency(self):
        counts = {}
        for item in self:
            counts.setdefault(item, 0)
            counts[item] += 1
        return counts

In [36]:
foo = FrequencyList('abacbad')
print('Length is ', len(foo))
foo.pop()
print('After pop:', repr(foo))
print('Frequency:', foo.frequency())

Length is  7
After pop: ['a', 'b', 'a', 'c', 'b', 'a']
Frequency: {'a': 3, 'b': 2, 'c': 1}


- Now,	imagine	you	want	to	**provide	an	object	that	feels	like	a	list,	allowing	indexing,	but isn’t	a	list	subclass.**	For	example,	say	you	want	to	provide	sequence	semantics	(like list	or	tuple)	for	a	binary	tree	class. 

In [37]:
class BinaryNode(object):
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

- How	do	you	make	this	act	like	a	sequence	type?	Python	implements	its	container behaviors	with	instance	methods	that	have	special	names.	When	you	access	a	sequence item	by	index:

In [38]:
bar = [1, 2, 3]
bar[0]

1

In [39]:
bar.__getitem__(0)

1

- To	make	the	BinaryNode	class	act	like	a	sequence,	you	can	provide	a	custom implementation	of	\_\_getitem\_\_	that	traverses	the	object	tree	depth	first. 

In [41]:
class IndexableNode(BinaryNode):
    def _search(self, count, index):
        # if self.value = count:
        pass
            
    def __getitem__(self, index):
        found , _ = self._search(0, index)
        if not found:
            raise IndexError('Index out of range')
        return found.value

In [42]:
tree = IndexableNode(
10, 
left=IndexableNode(
    5,
    left=IndexableNode(2),
    right=IndexableNode(
        6, right=IndexableNode(7))),
    right=IndexableNode(
        15, left=IndexableNode(11)))

- The	problem	is	that	implementing	\_\_getitem\_\_	isn’t	enough	to	provide	all	of	the sequence	semantics	you’d	expect. 

In [ ]:
len(tree)

- To	avoid	this	difficulty	throughout	the	Python	universe,	the	built-in	collections.abc module	defines	a	set	of	abstract	base	classes	that	provide	all	of	the	typical	methods	for each	container	type.	When	you	subclass	from	these	abstract	base	classes	and	forget	to implement	required	methods,	the	module	will	tell	you	something	is	wrong. 

In [ ]:
class SequenceNode(IndexableNode):
    def __len__(self):
        _, count = self._search(0, None)
        return count

In [ ]:
from collections.abc import Sequence

class BadType(Sequence):
    pass

In [43]:
foo = BadType()

NameError: name 'BadType' is not defined

- When	you	do	implement	all	of	the	methods	required	by	an	abstract	base	class,	as	I	did above	with	SequenceNode,	it	will	provide	all	of	the	additional	methods	like	index and	count	for	free. 

In [ ]:
class BetterNode(sequenceNode, Sequence):
    pass

tree = BetterNode(
    # ...
)

# I29 : Use Plain Attributes Instead of Get and Set Methods

In [44]:
class OldResistor(object):
    def __init__(self, ohms):
        self._ohms = ohms
        
    def get_ohms(self):
        return self._ohms
    
    def set_ohms(self, ohms):
        self._ohms = ohms

In [45]:
r0 = OldResistor(50e3)
print(r0.get_ohms())
print('After')
r0.set_ohms(10e3)
print(r0.get_ohms())

50000.0
After
10000.0


In [47]:
# clumsy
r0.set_ohms(r0.get_ohms()+5e3)

- In	Python,	however,	you	almost	never	need	to	implement	explicit	setter	or	getter	methods. 

In [48]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0

In [49]:
r1 = Resistor(50e3)
r1.ohms = 10e3

In [50]:
r1.ohms += 5e3

In [51]:
r1.ohms

15000.0

- Later,	if	you	decide	you	need	special	behavior	when	an	attribute	is	set,	you	can	migrate	to the	@property	decorator	and	its	corresponding	setter	attribute.	Here,	I	define	a	new subclass	of	Resistor	that	lets	me	vary	the	current	by	assigning	the	voltage property.	Note	that	in	order	to	work	properly	the	name	of	both	the	setter	and	getter methods	must	match	the	intended	property	name. 

In [52]:
class VoltageResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self._voltage = 0
    
    @property
    def voltage(self):
        return self._voltage
    
    @voltage.setter
    def voltage(self, voltage):
        self._voltage = voltage
        self.current = self._voltage / self.ohms

- Now,	assigning	the	voltage	property	will	run	the	voltage	setter	method,	updating	the current	property	of	the	object	to	match. 

In [53]:
r2 = VoltageResistance(1e3)
print('Before: %5r amps' % r2.current)
r2.voltage = 10
print('After: %5r amps' % r2.current)

Before:     0 amps
After:  0.01 amps


In [55]:
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError('%f ohms must be > 0' % ohms)
        self._ohms = ohms

In [56]:
r3 = BoundedResistance(1e3)

In [57]:
r3.ohms = 0

ValueError: 0.000000 ohms must be > 0

- This	happens	because	BoundedResistance.\_\_init\_\_	calls Resistor.\_\_init\_\_,	which	assigns	self.ohms	=	-5.	That	assignment	causes	the @ohms.setter	method	from	BoundedResistance	to	be	called,	immediately running	the	validation	code	before	object	construction	has	completed. 

- Make attr immutable

In [81]:
class FixedResistance(Resistor):
    # ...
    @property
    def ohms(self):
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        if hasattr(self, 'ohms'):
            raise AttributeError("Can't set attribute")
        self._ohms = ohms

In [83]:
r4 = FixedResistance(1e3)
# r4.ohms = 2e3
r4._ohms

1000.0

- The	biggest	shortcoming	of	@property	is	that	the	methods	for	an	attribute	can	only	be shared	by	subclasses.	Unrelated	classes	can’t	share	the	same	implementation.	However, Python	also	supports	descriptors

- Finally,	when	you	use	@property	methods	to	implement	setters	and	getters,	be	sure	that the	behavior	you	implement	is	not	surprising.	For	example,	don’t	set	other	attributes	in getter	property	methods. 

In [84]:
class Resistor(object):
    def __init__(self, ohms):
        self.ohms = ohms
        self.voltage = 0
        self.current = 0


class MysteriousResistor(Resistor):
    @property
    def ohms(self):
        self.voltage = self._ohms * self.current
        return self._ohms
    
    @ohms.setter
    def ohms(self, ohms):
        self._ohms = ohms
#     @ohms.setter
#     def ohms(self):
#         self.ohms = ohms


In [85]:
r7 = MysteriousResistor(10)
r7.current = 0.01
print(r7.voltage)
print('After')
r7.ohms
print(r7.voltage)

0
After
0.1


- The	best	policy	is	to	only	modify	related	object	state	in	@property.setter	methods. Be	sure	to	avoid	any	other	side	effects	the	caller	may	not	expect	beyond	the	object,	such	as importing	modules	dynamically,	running	slow	helper	functions,	or	making	expensive database	queries.	Users	of	your	class	will	expect	its	attributes	to	be	like	any	other	Python object:	quick	and	easy.	Use	normal	methods	to	do	anything	more	complex	or	slow. 

# I30 : Consider @property Instead of Refactoring Attributes

In [94]:
from datetime import timedelta
from datetime import datetime

class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.quota = 0
        
    def __repr__(self):
        return 'Bucket(quota=%d)' % self.quota

In [95]:
def fill(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount

In [96]:
def deduct(bucket, amount):
    now = datetime.now()
    if now - bucket.reset_time > bucket.period_delta:
        return False
    if bucket.quota- amount < 0:
        return False
    bucket.quota -= amount
    return True

In [97]:
bucket = Bucket(60)
fill(bucket, 100)
print(bucket)

Bucket(quota=100)


In [98]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print(bucket)

Had 99 quota
Bucket(quota=1)


In [99]:
if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
print(bucket)

Not enough for 3 quota
Bucket(quota=1)


In [108]:
class Bucket(object):
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
        
    def __repr__(self):
        return ('Bucket(max_quota=%d, quota_consumed=%d)' %
               (self.max_quota, self.quota_consumed))
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta

In [109]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
Filled Bucket(max_quota=100, quota_consumed=0)


In [110]:
if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
    
print('Now', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enough for 3 quota')
    
print('Still', bucket)

Had 99 quota
Now Bucket(max_quota=100, quota_consumed=99)
Not enough for 3 quota
Still Bucket(max_quota=100, quota_consumed=99)


# I31 : User Descriptors for Reusable @property Methods

In [113]:
class Homework(object):
    def __init__(self):
        self._grade = 0
        
    @property
    def grade(self):
        return self._grade
    
    @grade.setter
    def grade(self, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._grade = value

In [114]:
galileo = Homework()
galileo.grade = 95

In [117]:
class Exam(object):
    def __init__(self):
        self.writeing_grade = 0
        self._math_grade = 0
        
    @staticmethod
    def _check_grade(value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
            
    @property
    def writing_grade(self):
        return self._writing_grade
    
    @writing_grade.setter
    def writing_grade(self, value):
        self._check_grade(value)
        self._writing_grade = value
    
    @property
    def math_grade(self):
        return self.math_grade
    
    @math_grade.setter
    def math_grade(self, value):
        self.check_grade(value)
        self._math_grade = value

- Also, this approach is not general. If you want to reuse this percentage validation beyond homework and exams, you'd need to write the @property bolerplate and \_check\_grade repeatedly.

- The better way to do this in Python is to use a *descriptor*. The descriptor protocol defines how attributes access is interpreted by the language. A descriptor class  can provide \_\_get\_\_ and \_\_set\_\_ methods that let you reuse the grade validation behavior without any boilerplate.

In [119]:
class Grade(object):
    def __get__(*args, **kwargs):
        # ...
        pass
        
    def __set__(*args, **kwargs):
        #...
        pass
    
class Exam(object):
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [120]:
exam = Exam()
exam.writing_grade = 40

In [ ]:
# interpreted as 
Exam.__dict__['writing_grade'].__set__(exam, 40)

In [121]:
print(exam.writing_grade)

None


In [ ]:
# interpreted as 
print(Exam.__dict__['writing_grade'].__get__(exam, Exam))

- What	drives	this	behavior	is	the	\_\_getattribute\_\_	method	of	object

In [124]:
class Grade(object):
    def __init__(self):
        self._value = 0
        
    def __get__(self, instance, instance_type):
        return self._value

    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._value = value
        
    
class Exam(object):
    # Class attributes
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [125]:
first_exam = Exam()
first_exam.writing_grade = 82
first_exam.science_grade = 99
print('Writing', first_exam.writing_grade)
print('Science', first_exam.science_grade)

Writing 82
Science 99


- Works but on ..

In [126]:
second_exam = Exam()
second_exam.writing_grade = 75
print('Second', second_exam.writing_grade, 'is right')
print('Fist', first_exam.writing_grade, 'is_wrong')

Second 75 is right
Fist 75 is_wrong


- The	problem	is	that	a	single	Grade	instance	is	shared	across	all	Exam	instances	for	the class	attribute	writing\_grade.	The	Grade	instance	for	this	attribute	is	constructed once	in	the	program	lifetime	when	the	Exam	class	is	first	defined,	not	each	time	an	Exam instance	is	created. 

In [127]:
class Grade(object):
    def __init__(self):
        self._values = {}
        
    def __get__(self, instance, instance_type):
        if instance is None: return self
        return self._values.get(instance, 0)
    
    def __set__(self, instance, value):
        if not (0 <= value <= 100):
            raise ValueError('Grade must be between 0 and 100')
        self._values[instance] = value

- This	implementation	is	simple	and	works	well,	but	there’s	still	one	gotcha:	It	leaks memory.	The	\_values	dictionary	will	hold	a	reference	to	every	instance	of	Exam	ever passed	to	\_\_set\_\_	over	the	lifetime	of	the	program.	This	causes	instances	to	never	have their	reference	count	go	to	zero,	preventing	cleanup	by	the	garbage	collector. 

- To fix this, I can use Python's weakref built-in module. this module provides a special class called WeakKeyDictionary that can take the place of the simple dictionary used for \_values. The uniuqe behavior of WeakKeyDictionary is that it will remove Exam instances from its set of keys when the runtime known it's holding the instance's last remaining reference in the program. Python will do the bookkeeping for you and ensure that the \_values dictionary will be empty whne all Exam instances are no longer in use.

In [129]:
import weakref 

class Grade(object):
    def __init__(self):
        self._values = weakref.WeakKeyDictionary()
    #.. 
    
    
class Exam(object):
    math_grade = Grade()
    writing_grade = Grade()
    science_grade = Grade()

In [130]:
first_exam = Exam()
first_exam.writing_grade = 82
second_exam = Exam()
second_exam.writing_grade = 75

# I32 : Use \_\_getattr\_\_, \_\_getattribute\_\_, and \_\_setattr\_\_ for Lazy Attributes